# Approach to be followed - 
1. Get all 'Ensembl homo_sapiens gene IDs' by performing string manipulation on the gff3 file.
2. Pass all the 'gene IDs' (in batches of 100) to REST API to get responses in json format.
3. Extract the required data (start, end, strand, ...) from the json response.

###### -----------------------------------------------------------------------------------------------------------------------------------------------------

## Use gffpandas python library to load gff3 file and get data in 'pandas dataframe format'

In [1]:
import gffpandas.gffpandas as gffpd
path = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_sequence/Homo_sapiens.GRCh38.96.gff3'

In [2]:
%%time
annotation = gffpd.read_gff3(path)

/Users/pankajverma/anaconda3/envs/gsoc_ensembl/lib/python3.6/site-packages/gffpandas/gffpandas.py:35: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  "attributes"])
/Users/pankajverma/anaconda3/envs/gsoc_ensembl/lib/python3.6/site-packages/ipykernel/__main__.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if __name__ == '__main__':


CPU times: user 6.58 s, sys: 585 ms, total: 7.17 s
Wall time: 7.25 s


In [1]:
#print(annotation.header)

In [2]:
#print(annotation.df)

In [3]:
#print(annotation.df['attributes'][16:29])

In [4]:
#print(annotation.df['attributes'][16])

## 'attributes' column (of the gff3 file) contains all the gene IDs

In [5]:
annotation.df['attributes'][16][0:8]
gene_ids = []

## Down below - to get all gene IDs (by performing string manipulation)

In [20]:
%%time
count = 0
for i in range(len(annotation.df['attributes'])):
    if annotation.df['attributes'][i][0:8] == 'ID=gene:':
        count = count + 1
        gene_ids.append(annotation.df['attributes'][i][8:23])
print(count)        

58884
CPU times: user 1min 43s, sys: 201 ms, total: 1min 43s
Wall time: 1min 44s


In [60]:
import tqdm

In [22]:
gene_ids.sort() #arranging the ids in ascending order

## Given below 2 cells for recording the log of responses from API (to keep a record of the error received)

In [6]:
import csv 
header = ['OK Batch No', 'OK Batch Response']
# ok_batch_no = []
# ok_batch_response = []
# error_batch_no = []
# error_batch_response = []
ok_response_path = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/api_response_record/ok_responses.csv'

with open(ok_response_path, 'wt', newline ='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(i for i in header)

In [7]:
import csv 
import pandas as pd
header = ['Error Batch No', 'Error Batch Response']
# ok_batch_no = []
# ok_batch_response = []
# error_batch_no = []
# error_batch_response = []
error_response_path = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/api_response_record/error_responses.csv'

with open(error_response_path, 'wt', newline ='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(i for i in header)

## Below cell - saving all gene ids in a csv file, and then loading them again

In [38]:
import csv 
header = ['SNO', 'All gene ID']

path = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/all_gene_ids.csv'

with open(path, 'wt', newline ='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(i for i in header)

In [39]:
s_no = []
for i in range(len(gene_ids)):
    s_no.append(i+1)


In [43]:
len(s_no)

58884

In [50]:
import pandas as pd

df = pd.read_csv(path)

df[df.columns[0]] = s_no
df[df.columns[1]] = gene_ids

df.to_csv(path)

In [8]:
import csv 
import pandas as pd
import numpy as np
import json

In [34]:
path = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/all_gene_ids.csv'

In [35]:
df = pd.read_csv(path)
gene_ids = df['All gene ID'].tolist()

In [36]:
len(gene_ids)
test = []

In [37]:
test = gene_ids

## total gene IDs (for homo_sapiens) are : 58884

In [38]:
len(test)

58884

## below cell to check if the IDs are in the correct format

In [33]:
c = 0
d = 0
for i in range(len(gene_ids)):
    if test[i][0:9] == 'ENSG00000':
        c = c + 1
    num = test[i][4:16]
    num = int(a)
    if isinstance(num, int):
        d = d + 1
print(c)  
print(d)  

58884
58884


In [39]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import requests, sys
import json, urllib
import time
from tqdm import tqdm
def csv_to_id(batch):
    cleaned = json.dumps(batch)        
    correct_format = "{" +'"ids": ' + cleaned + "}"
    return correct_format

In [40]:
cleaned_ids = []
cleaned_ids = gene_ids
ok_batch_no = []
ok_batch_response = []
error_batch_no = []
error_batch_response = []
len(cleaned_ids)


58884

### 1. Sending the gene IDs to ensembl API (in batches of 100 [which 1/10th of the assigned limit for post response])
### 2. Reason for sending in batches of 100 (although the limit is 1000) - because it was taking a long time for getting data for 1000 IDs. 
### 3. Total time taken to receive responses for 58884 IDs = ~ 2 hours 45 min (on mac CPU - 2.7 GHz Intel Core i5)
### 4. Responses received in 589 json files (having 100 responses in each file)

In [6]:
%%time
for i in tqdm(range(0, len(test), 100)):
    batch = test[i:i + 100]
    cleaned_IDs = csv_to_id(batch)
    
    server = "https://rest.ensembl.org"
    ext = '/lookup/id/?format=full;expand=1;utr=1;phenotypes=1'
    #ext = '/lookup/id/?
    headers = {'Content-Type' : 'application/json', "Accept" : 'application/json'}
    #'{"ids" : ["ENSG00000255689", "ENSG00000254443"]}'
    #cleaned_IDs = {"ids": ["ENSG00000255689", "ENSG00000254443"]}
    r = requests.post(server+ext,headers=headers, data='{0}'.format(cleaned_IDs))
    #time.sleep(10)
#     print(str(r))
#     print(type(r))
    if str(r) == '<Response [200]>':
        df = pd.read_csv(ok_response_path)
        ok_batch_no = i
        ok_batch_response = str(r)
        df2 = df.append(pd.DataFrame([[ok_batch_no, ok_batch_response]], columns=df.columns))
        df2.to_csv(ok_response_path, index=False)
#         ok_batch_no.append(i)
#         ok_batch_response.append(str(r))
        
    else:
        df = pd.read_csv(error_response_path)
        error_batch_no = i
        error_batch_response = str(r)
        df2 = df.append(pd.DataFrame([[error_batch_no, error_batch_response]], columns=df.columns))
        df2.to_csv(error_response_path, index=False)
        continue
#         error_batch_no.append(i)
#         error_batch_response.append(str(r))
    decoded = r.json()

# below 2 lines not advisable since it will take a lot of time for json file to load once it becomes large    
#     with open('/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/test_all_gene_data.json', 'a') as outfile:
#         json.dump(decoded, outfile, indent=4, sort_keys=True) 
        
    with open('/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/json_all_gene_data(batches)/batch_'+str(i)+'.json', 'w') as outfile:
        json.dump(decoded, outfile, indent=4, sort_keys=True) 
        
#print(repr(decoded))

## Merging all the json files into one json file (list format)

In [7]:
%%time
output_list = []
for i in tqdm(range(0,len(test),100)):
    
    in_path = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/json_all_gene_data(batches)/'
    batch_json = in_path + 'batch_' + str(i) + '.json'
    
    with open(batch_json) as infile:
        output_list.append(json.load(infile))

out_path = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/'

with open(out_path + "merged_all_gene_data.json", "w") as outfile:
    json.dump(output_list, outfile, indent=4, sort_keys=True)
        #json.dump(output_list, outfile)

## Merging all the json files into one json file (dict format) 
### Saving in dict format is better for future reference

In [8]:
%%time
path_1 = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/json_all_gene_data(batches)/batch_0.json'

with open(path_1) as infile:
    dic1 = json.load(infile)

#dic1 = json.load(path_1)
for i in tqdm(range(100,len(test),100)):
    in_path = '/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/json_all_gene_data(batches)/'
    batch_json = in_path + 'batch_' + str(i) + '.json'
    
    with open(batch_json) as infile:
        dic2 = json.load(infile)
    #dic2 = json.load(batch_json)
    dic1.update(dic2)

## dumping data into one json file

In [9]:
%%time
with open("/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/merged_all_gene_data(dict).json", "w") as outfile:
    json.dump(dic1, outfile, indent=4, sort_keys=True)

## calculating time taken to open 1.05GB json file

In [74]:
%%time
with open("/Users/pankajverma/Desktop/summerInternship2k17/GSoC/Ensembl/1.lncRNA/dataset/Ensembl_numeric/merged_all_gene_data(dict).json") as access_json:
    read_content = json.load(access_json)

CPU times: user 24.1 s, sys: 47.2 s, total: 1min 11s
Wall time: 1min 20s
